In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt
import pylab as pl
import os
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
#training data set
train=pd.read_csv(r'C:\Users\hp\Documents\Projects\Titanic Modal Prediction\Input Data/train.csv');
print(train.head())
print(train.describe())
print(train.dtypes)
#plt.scatter(train.PassengerId,train.Survived)
#plt.xlabel("PassengerId");
#plt.ylabel("Survived");
#plt.show();
mean_train=train["Age"].mean()
print("mean of the training set: ",mean_train)
train["Age"].replace(np.nan,mean_train,inplace=True)
Embarked_dummy_train=pd.get_dummies(train["Embarked"])
train=pd.concat([train,Embarked_dummy_train],axis=1)
Sex_dummy_train=pd.get_dummies(train["Sex"])
train=pd.concat([train,Sex_dummy_train],axis=1)
Pclass_dummy_train=pd.get_dummies(train["Pclass"])
train=pd.concat([train,Pclass_dummy_train],axis=1)
train["Cabin"].fillna("Z", inplace=True)
train["Cabin"] = train["Cabin"].astype("str").str[0].astype("category")
Cabin_dummy_train = pd.get_dummies(train["Cabin"])
train=pd.concat([train,Cabin_dummy_train],axis=1)
train.drop(train[["Embarked","Sex","Pclass","Cabin","Name","Ticket"]],axis=1,inplace=True)
train.rename(columns={1:"Pclass_1",2:'Pclass_2',3:'Pclass_3'},inplace=True)
print(train.columns)
print("columns with the missing value: ",train.columns[train.isnull().any()])
#normalization
#from sklearn import preprocessing
#train= preprocessing.StandardScaler().fit(train).transform(train)
#train[0:5]

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
  

In [3]:
#test data set
test=pd.read_csv(r'C:\Users\hp\Documents\Projects\Titanic Modal Prediction\Input Data\test.csv')
print(test.head())
print(test.describe())
print(test.dtypes)
#plt.scatter(test.PassengerId,test.Survived)
#plt.xlabel("PassengerId")
#plt.ylabel("Survived")
#plt.show()
print("columns with the missing values: ",test.columns[test.isnull().any()])
#test.dropna(subset=["Age"],axis=0)
mean_age_test=test["Age"].mean()
print("Mean of age of test set: ",mean_age_test)
test["Age"].replace(np.nan,mean_age_test,inplace=True)
mean_fare_test=test["Fare"].mean()
print("Mean of fare of test data: ",mean_fare_test)
test["Fare"].replace(np.nan,mean_fare_test,inplace=True)
Embarked_dummy_test=pd.get_dummies(test["Embarked"])
test=pd.concat([test,Embarked_dummy_test],axis=1)
Sex_dummy_test=pd.get_dummies(test["Sex"])
test=pd.concat([test,Sex_dummy_test],axis=1)
Pclass_dummy_test=pd.get_dummies(test["Pclass"])
test=pd.concat([test,Pclass_dummy_test],axis=1)
test["Cabin"].fillna("Z", inplace=True)
test["Cabin"] = test["Cabin"].astype("str").str[0].astype("category")
Cabin_dummy_test = pd.get_dummies(test["Cabin"])
test=pd.concat([test,Cabin_dummy_test],axis=1)
test.drop(test[["Embarked","Sex","Pclass","Cabin","Name","Ticket"]],axis=1,inplace=True)
test.rename(columns={1:"Pclass_1",2:'Pclass_2',3:'Pclass_3'},inplace=True)
print(test.columns)
print("columns with the missing values: ",test.columns[test.isnull().any()])
#normalizing the test set
#from sklearn import preprocessing
#test = preprocessing.StandardScaler().fit(test).transform(test)
#test[0:5]

   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S  
       PassengerId      Pclass         Age       SibSp       Parch        Fare
count   418.000000  418.000000  332.000000  418.000000  418.000000  417.000000
mean   1100.500000 

In [ ]:
#Normalization
#from sklearn import preprocessing
#train= preprocessing.StandardScaler().fit(train).transform(train)
#print(train[0:5])
#test=preprocessing.StandardScaler().fit(test).transform(test)
#print(test[0:5])

In [4]:
train_x=np.asarray(train[['PassengerId','SibSp','Parch','Age','Fare','C','S','Q','Pclass_1','Pclass_2','Pclass_3','A','B','C','D','E','F','G','Z','male','female']]);
train_y=np.asarray(train['Survived']);
print("train set ",train_x.shape,train_y.shape)

train set  (891, 23) (891,)


In [5]:
test_x=np.asarray(test[['PassengerId','SibSp','Parch','Age','Fare','C','S','Q','Pclass_1','Pclass_2','Pclass_3','A','B','C','D','E','F','G','Z','male','female']]);
#test_y=np.asarray(test['Survived']);
print("test set ",test_x.shape)

test set  (418, 23)


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(train_x,train_y)
LR

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [7]:
yhat=LR.predict(test_x)
print("shape of Yhat: ",yhat.shape)
print(yhat)


shape of Yhat:  (418,)
[0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 0 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 1 0
 1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 1 0 1 0 1 0 0 1 0 0 0]


In [11]:
sub_df = pd.read_csv(r"C:\Users\hp\Documents\Projects\Titanic Modal Prediction\Input Data/gender_submission.csv")
sub_df.Survived = yhat
sub_df.to_csv(r"C:\Users\hp\Documents\Projects\Titanic Modal Prediction\Output\submission.csv", index=False)

In [12]:
#from sklearn.metrics import jaccard_similarity_score
#jaccard_similarity_score(test_y, yhat)

In [13]:
#it tells the probabilty to belong to class 0 and class 1 for each test case
yhat_prob = LR.predict_proba(test_x)
print(yhat_prob.shape)
yhat_prob

(418, 2)


array([[0.77724976, 0.22275024],
       [0.66871423, 0.33128577],
       [0.79775986, 0.20224014],
       [0.7704411 , 0.2295589 ],
       [0.54931197, 0.45068803],
       [0.72420354, 0.27579646],
       [0.53014247, 0.46985753],
       [0.6839626 , 0.3160374 ],
       [0.44810242, 0.55189758],
       [0.77462216, 0.22537784],
       [0.78217762, 0.21782238],
       [0.72963991, 0.27036009],
       [0.24029719, 0.75970281],
       [0.81312731, 0.18687269],
       [0.37338294, 0.62661706],
       [0.36949393, 0.63050607],
       [0.69946491, 0.30053509],
       [0.70954557, 0.29045443],
       [0.57985413, 0.42014587],
       [0.57108269, 0.42891731],
       [0.6725152 , 0.3274848 ],
       [0.72149784, 0.27850216],
       [0.3993313 , 0.6006687 ],
       [0.48331613, 0.51668387],
       [0.05398132, 0.94601868],
       [0.84797392, 0.15202608],
       [0.21286565, 0.78713435],
       [0.71489878, 0.28510122],
       [0.66144935, 0.33855065],
       [0.76997561, 0.23002439],
       [0.